# Neural networks code and tests notebook

# This notebook takes the flux and photon index data and try to predict classes from this data on a :

# - Conv1D network

In [1]:
import numpy as np
from numpy import random
import pandas as pd
import re
import os
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import math
from scipy import interpolate
import sys 
from re import search
from astropy.io import fits
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude
import shutil
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import glob
from datetime import datetime
from keras.layers import Conv1D, Dense, Dropout, Input, Concatenate, GlobalMaxPooling1D, BatchNormalization,GlobalAveragePooling1D
from keras.layers.convolutional import MaxPooling1D
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,LSTM
from keras.utils import to_categorical
from keras.layers import Dense, Activation, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit # or StratifiedShuffleSplit
from pandas import DataFrame


In [2]:
pathON="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/ON_data/"
pathOFF="C:/Users/pierr/Documents_kanop/Shps/Pierre_points_for_louis/Bureau/interpo_ONOFF/OFF_data/"

In [3]:
#Storing in a list the adresses of all the data files in the notebook

data_files_ON=[]
file_names_ON=[]

data_files_OFF=[]
file_names_OFF=[]
data_files_ALL=[]
file_names_ALL=[]

#Storing in a list the adresses of all the data files in the notebook
for filename in os.listdir(pathOFF):
    f = os.path.join(pathOFF,filename)
    if os.path.isfile(f):
        data_files_OFF.append(f)
        file_names_OFF.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)
for filename in os.listdir(pathON):
    f = os.path.join(pathON,filename)
    if os.path.isfile(f):
        data_files_ON.append(f)
        file_names_ON.append(filename)
        data_files_ALL.append(f)
        file_names_ALL.append(filename)        

In [4]:
#Using data_files with only the 36 binning
Filter=True
binning=37


if Filter==True:
    
    idx_OFF=[]
    idx_ON=[]
    dataON=[]
    dataOFF=[]
    for i in range(len(data_files_OFF)):
        dataframe = pd.read_csv(data_files_OFF[i])
        lg = len(dataframe)
        
        if lg==binning:
            idx_OFF.append(i)
    
    for i in range(len(data_files_ON)):
        dataframe = pd.read_csv(data_files_ON[i])
        lg = len(dataframe)
#         print(lg)
        if lg==binning:
            idx_ON.append(i)

    #Future input shape : 3511 sources, 2 features (as time series) , x binnings
    for i in range(len(idx_OFF)):

        a=idx_OFF[i]
        dataOFF.append(data_files_OFF[a])
    for i in range(len(idx_ON)):

        a=idx_ON[i]
        dataON.append(data_files_ON[a])

idx = idx_OFF+idx_ON


In [5]:
#3D shape of data, multivariate time series

lgON=len(dataON)
lgOFF=len(dataOFF)
lg=lgON+lgOFF
Labels = []


nbfeatures=2

a=binning
b=nbfeatures
c=lg

# multivariate
y=np.zeros((2,lg))
bigdata= np.zeros((b,c,a))


for j in range(len(dataOFF)):

    dataframe=pd.read_csv(dataOFF[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][j]=dataframe['Flux']
    bigdata[1][j]=dataframe['Photon Index']

for j in range(len(dataON)):

    v=j+len(dataOFF)
    dataframe=pd.read_csv(dataON[j],index_col=[0])
    dataframe.columns=['MJD','Flux','Delta Flux','Photon Index','Delta Index','TS','fratio']

    bigdata[0][v]=dataframe['Flux']
    bigdata[1][v]=dataframe['Photon Index']



for i in range(lgOFF):
    
    Labels.append(int(0))
    
for i in range(lgON):
    
    Labels.append(int(1))


In [6]:
#Reshaping data matrix
bigdata=bigdata.reshape(c,nbfeatures,binning)

In [29]:
from keras.layers import Conv1D
from sklearn.metrics import f1_score
ON_accuracy=[]
OFF_accuracy=[]
fscore=[]
FPOS=[]
FNEG= []
#Measuring the meaned accuracy of  correct prediction of labellisation for each class  over a defined number of steps
for i in range(200,800):
    nb_filters=20
    


    weight_for_0 = (1 / lgOFF) * (lg / 2.0)
    weight_for_1 = (1 / lgON) * (lg / 2.0)
    callback=tensorflow.keras.callbacks.EarlyStopping(monitor='binary_accuracy',patience=2, verbose=0, mode='auto',baseline=None, restore_best_weights=True)
    class_weight = {0: weight_for_0, 1: weight_for_1}
    bigdata=bigdata.reshape(c,binning,nbfeatures)
    #Conv1D Architecture
    model = Sequential()

    model.add(Conv1D(nb_filters, 1, padding="same", activation="tanh",input_shape=(binning,nbfeatures)))
    model.add(Conv1D(15, 1, padding="same", activation="tanh",input_shape=(nb_filters,nbfeatures)))
    model.add(Conv1D(9, 1, padding="same", activation="tanh",input_shape=(15,nbfeatures)))
    model.add(GlobalMaxPooling1D())
    
    model.add(Dense(9, activation='tanh'))
    
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=tensorflow.keras.losses.BinaryCrossentropy(reduction='sum'),optimizer='SGD', metrics=['binary_accuracy'])


    x_train, x_test, y_train, y_test = train_test_split(bigdata, Labels, test_size=0.1, random_state=i)
    y_test2=y_test.copy()
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    #fitting data
    model.fit(x_train, y_train,epochs=4,batch_size=1,class_weight=class_weight,callbacks=callback)
    print('Iteration no.',i)
    #Obtain the accuradcy of prediction for each class
    prediction= model.predict(x_test)
    predicted_labels=[]
    for i in range(len(prediction)):
        if prediction[i][0]>0.5:
            predicted_labels.append(0)
        else:
            predicted_labels.append(1)
    on_score=0
    on_nbs=0
    off_nbs=0
    off_score=0
    foff=0
    fon=0
    for i in range(len(y_test)):
        if y_test2[i]==1 :
            on_nbs+=1
            if predicted_labels[i]==1:
                on_score+=1
            else : 
                fon+=1
        if y_test2[i]==0 :
            off_nbs+=1
            if predicted_labels[i]==0:
                off_score+=1 
            else:
                foff+=1
    if on_nbs>0:    
        ON_accuracy.append(100*(on_score/on_nbs))
        FPOS.append(100*(fon/on_nbs))
    OFF_accuracy.append(100*(off_score/off_nbs))
    FNEG.append(100*(foff/off_nbs))
    f1= f1_score(y_test2,predicted_labels,average='weighted')
    fscore.append(f1)
print("Accuracy for ON class: ",np.mean(ON_accuracy) ,"%")
print("Accuracy for OFF class: ",np.mean(OFF_accuracy) ,"%")
print("False Positive rate: ",np.mean(FPOS) ,"%")
print("False Negative rate: ",np.mean(FNEG) ,"%")
print("F1 score: ",np.mean(fscore))

Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.6097 - binary_accuracy: 0.5362
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5401 - binary_accuracy: 0.5133
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5150 - binary_accuracy: 0.5133
Iteration no. 200
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6242 - binary_accuracy: 0.4888
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5304 - binary_accuracy: 0.5592
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5211 - binary_accuracy: 0.5184
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5102 - binary_accuracy: 0.5184
Iteration no. 201
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.6074 - binary_accuracy: 0.6551
Epoch 2/4
980/

980/980 [==============================] - 1s 2ms/step - loss: 0.5408 - binary_accuracy: 0.5184
Epoch 4/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5250 - binary_accuracy: 0.5429
Iteration no. 236
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5934 - binary_accuracy: 0.5296
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5391 - binary_accuracy: 0.5816
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5261 - binary_accuracy: 0.6582
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5162 - binary_accuracy: 0.5173
Iteration no. 237
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6297 - binary_accuracy: 0.7668
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5723 - binary_accuracy: 0.5582
Epoch 3/4
980/980 [=====

980/980 [==============================] - 2s 2ms/step - loss: 0.4960 - binary_accuracy: 0.5255
Iteration no. 255
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.6257 - binary_accuracy: 0.5832
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5563 - binary_accuracy: 0.5276
Epoch 3/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5219 - binary_accuracy: 0.5582
Iteration no. 256
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5792 - binary_accuracy: 0.6051
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4893 - binary_accuracy: 0.5653
Epoch 3/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4621 - binary_accuracy: 0.5663
Iteration no. 257
4/4 [==============================] - 0s 3ms/step
Epoch 1/4
980/980 [==============================] - 3s 2ms/s

980/980 [==============================] - 1s 1ms/step - loss: 0.5235 - binary_accuracy: 0.5408
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5124 - binary_accuracy: 0.5153
Iteration no. 274
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5612 - binary_accuracy: 0.7102
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5227 - binary_accuracy: 0.5133
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4947 - binary_accuracy: 0.5653
Iteration no. 275
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6576 - binary_accuracy: 0.5158
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5566 - binary_accuracy: 0.5143
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5459 - binary_accuracy: 0.5163
Epoch 4/4
980/980 [=====

980/980 [==============================] - 1s 1ms/step - loss: 0.5209 - binary_accuracy: 0.5224
Iteration no. 293
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6092 - binary_accuracy: 0.5969
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5441 - binary_accuracy: 0.5367
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5142 - binary_accuracy: 0.5224
Iteration no. 294
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6770 - binary_accuracy: 0.6949
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5789 - binary_accuracy: 0.5112
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5589 - binary_accuracy: 0.5184
Iteration no. 295
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/s

...

980/980 [==============================] - 1s 1ms/step - loss: 0.4737 - binary_accuracy: 0.5755
Iteration no. 669
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6173 - binary_accuracy: 0.6648
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5514 - binary_accuracy: 0.5347
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5251 - binary_accuracy: 0.5286
Iteration no. 670
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6349 - binary_accuracy: 0.6597
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5321 - binary_accuracy: 0.5816
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5152 - binary_accuracy: 0.5265
Iteration no. 671
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/s

980/980 [==============================] - 1s 1ms/step - loss: 0.5501 - binary_accuracy: 0.5163
Iteration no. 707
4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6213 - binary_accuracy: 0.5439
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5183 - binary_accuracy: 0.5276
Epoch 3/4
980/980 [==============================] - 2s 2ms/step - loss: 0.4940 - binary_accuracy: 0.5133
Iteration no. 708
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6282 - binary_accuracy: 0.5372
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5343 - binary_accuracy: 0.5122
Epoch 3/4
980/980 [==============================] - 1s 2ms/step - loss: 0.5254 - binary_accuracy: 0.5112
Iteration no. 709
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/s

980/980 [==============================] - 1s 1ms/step - loss: 0.5334 - binary_accuracy: 0.5276
Iteration no. 726
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5917 - binary_accuracy: 0.6184
Epoch 2/4
980/980 [==============================] - 2s 2ms/step - loss: 0.5085 - binary_accuracy: 0.5286
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4875 - binary_accuracy: 0.5265
Iteration no. 727
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.5919 - binary_accuracy: 0.5112
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5329 - binary_accuracy: 0.5194
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5071 - binary_accuracy: 0.5561
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.4955 - binary_accuracy: 0.5622
Iteration no. 728
4/4 [=

4/4 [==============================] - 0s 1ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6228 - binary_accuracy: 0.6041
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5228 - binary_accuracy: 0.5673
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5139 - binary_accuracy: 0.5439
Iteration no. 746
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6124 - binary_accuracy: 0.5740
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5204 - binary_accuracy: 0.5214
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5089 - binary_accuracy: 0.5327
Iteration no. 747
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6107 - binary_accuracy: 0.5974
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - lo

980/980 [==============================] - 2s 1ms/step - loss: 0.6246 - binary_accuracy: 0.5367
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5491 - binary_accuracy: 0.5663
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5541 - binary_accuracy: 0.5306
Epoch 4/4
980/980 [==============================] - 1s 2ms/step - loss: 0.5363 - binary_accuracy: 0.5143
Iteration no. 783
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [==============================] - 2s 1ms/step - loss: 0.6148 - binary_accuracy: 0.5204
Epoch 2/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5166 - binary_accuracy: 0.5153
Epoch 3/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5117 - binary_accuracy: 0.6051
Epoch 4/4
980/980 [==============================] - 1s 1ms/step - loss: 0.5073 - binary_accuracy: 0.5112
Iteration no. 784
4/4 [==============================] - 0s 2ms/step
Epoch 1/4
980/980 [=====